In [9]:
import tensorflow as tf
import numpy as np
import cv2
import input_data

In [10]:
img_height=480
img_width=640
#Network Parameters
n_input=img_height*img_width*3
learning_rate=8e-4
training_iters=10000
batch_size=2
display_step=1

#tf graph input
x=tf.placeholder(tf.float32,[None,img_height,img_width,3])
y=tf.placeholder(tf.float32,[None,img_height,img_width,16])

In [11]:
'''def getTestPicArray(filename) :
    im = cv2.imread(filename)
    #im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_arr = np.array(im)
    nm = im_arr.reshape((1, n_input))
    nm = nm.astype(np.float32)
    return nm'''

'def getTestPicArray(filename) :\n    im = cv2.imread(filename)\n    #im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)\n    im_arr = np.array(im)\n    nm = im_arr.reshape((1, n_input))\n    nm = nm.astype(np.float32)\n    return nm'

In [12]:
def conv2d(img,w,b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def max_pool(img,k):
    return tf.nn.max_pool(img,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
def conv2d_transpose(img,w,b,outputShape):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d_transpose(img,w,output_shape=outputShape,strides=[1,1,1,1],padding='SAME'),b))

In [13]:
wc1=tf.Variable(tf.random_normal([5,5,3,16]),name="wc1")
bc1=tf.Variable(tf.random_normal([16]),name="bc1")

wc2=tf.Variable(tf.random_normal([5,5,16,64]),name="wc2")
bc2=tf.Variable(tf.random_normal([64]),name="bc2")

wc3=tf.Variable(tf.random_normal([5,5,64,64]),name="wc3")
bc3=tf.Variable(tf.random_normal([64]),name="bc3")

wc4=tf.Variable(tf.random_normal([5,5,16,64]),name="wc4")
bc4=tf.Variable(tf.random_normal([16]),name="bc4")

wc5=tf.Variable(tf.random_normal([5,5,16,16]),name="wc5")
bc5=tf.Variable(tf.random_normal([16]),name="bc5")

#wout=tf.Variable(tf.random_normal([1,img_height,img_width,16]),name="wout")
#bout=tf.Variable(tf.random_normal([1,img_height,img_width,16]),name="bout")

In [14]:
#Construct model
_X=tf.reshape(x,shape=[-1,img_height,img_width,3])
conv1_1=conv2d(_X,wc1,bc1)
conv1_2=max_pool(conv1_1,k=2)

conv2_1=conv2d(conv1_2,wc2,bc2)
conv2_2=max_pool(conv2_1,k=2)

conv3=conv2d(conv2_2,wc3,bc3)
conv3=max_pool(conv3,k=1)

add1=tf.add(conv2_2,conv3)
#add1=conv2+conv3

conv_t1=tf.image.resize_bilinear(add1,conv2_1.get_shape().as_list()[1:3])
tmpShape=conv1_2.get_shape().as_list()
tmpShape[0]=batch_size
conv_t1=conv2d_transpose(conv_t1,wc4,bc4,outputShape=tmpShape)

add2=tf.add(conv1_2,conv_t1)

conv_t2=tf.image.resize_bilinear(add2,conv1_1.get_shape().as_list()[1:3])
tmpShape=_X.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=16
conv_t2=conv2d_transpose(conv_t2,wc5,bc5,outputShape=tmpShape)

#predict=tf.add(tf.multiply(conv_t2,wout),bout)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=conv_t2,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [15]:
init=tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [16]:
with tf.Session(config=config) as sess:
    saver = tf.train.Saver({"wc1":wc1,"bc1":bc1,"wc2":wc2,"bc2":bc2,"wc3":wc3,"bc3":bc3,"wc4":wc4,"bc4":bc4,"wc5":wc5,"bc5":bc5})
    sess.run(init)
    step=0
    now_at=0
    while step*batch_size<training_iters:
        X,Y=input_data.next_batch(
                img_dir_path='mpii_human_pose_v1\\output_images\\',
                index_path='train_data\\new_data.json',
                img_height=480,img_width=640,
                batch_size=batch_size)
        sess.run(optimizer,feed_dict = {x:X,y:Y})
        loss=sess.run(cost,feed_dict = {x:X,y:Y})
        if step%display_step==0:
            print("Iter "+str(step*batch_size)+", Minibatch Loss="+"{:.6f}".format(loss))
        step+=1
        now_at+=batch_size
    save_path = saver.save(sess, "model/model.ckpt")
print("done!")


Start new batch!
Iter 0, Minibatch Loss=1522775168.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 18, Minibatch Loss=2065979264.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 36, Minibatch Loss=2744016384.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 54, Minibatch Loss=3596646656.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 72, Minibatch Loss=4793545216.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 90, Minibatch Loss=6560465408.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 108, Minibatch Loss=9129753600.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new

Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1008, Minibatch Loss=270223387131904.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1026, Minibatch Loss=299177691054080.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1044, Minibatch Loss=330547897303040.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1062, Minibatch Loss=364486460440576.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1080, Minibatch Loss=401159172915200.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1098, Minibatch Loss=440732095610880.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1116

Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1980, Minibatch Loss=10732196633509888.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 1998, Minibatch Loss=11251137130790912.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 2016, Minibatch Loss=11789460815478784.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 2034, Minibatch Loss=12347567119532032.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 2052, Minibatch Loss=12925727699632128.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 2070, Minibatch Loss=13523195231469568.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batc

Iter 2934, Minibatch Loss=75474171802419200.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 2952, Minibatch Loss=77770046570496000.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 2970, Minibatch Loss=80119582659969024.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 2988, Minibatch Loss=82523707783774208.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3006, Minibatch Loss=84983186446090240.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3024, Minibatch Loss=87498757381292032.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3042, Minibatch Loss=90071082014343168.000000
Start new batch!
Start new batc

Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3906, Minibatch Loss=298766446324875264.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3924, Minibatch Loss=305297339235434496.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3942, Minibatch Loss=311935297090748416.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3960, Minibatch Loss=318681659920613376.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3978, Minibatch Loss=325537905193779200.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 3996, Minibatch Loss=332505098062135296.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start ne

...

Start new batch!
Start new batch!
Start new batch!
Iter 5814, Minibatch Loss=1833603727156576256.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 5832, Minibatch Loss=1858749145766952960.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 5850, Minibatch Loss=1884146489979043840.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 5868, Minibatch Loss=1909797134182383616.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 5886, Minibatch Loss=1935702315327553536.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 5904, Minibatch Loss=1961864232437809152.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 5922, Minibatc

Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 6768, Minibatch Loss=3550879124094451712.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 6786, Minibatch Loss=3591591290891927552.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 6804, Minibatch Loss=3632641832392851456.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 6822, Minibatch Loss=3674032122986758144.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 6840, Minibatch Loss=3715764086818996224.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 6858, Minibatch Loss=3757837998767472640.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
It

Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 7722, Minibatch Loss=6215577714724700160.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 7740, Minibatch Loss=6276642391508123648.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 7758, Minibatch Loss=6338140275872890880.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 7776, Minibatch Loss=6400073017086443520.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 7794, Minibatch Loss=6462444463439478784.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 7812, Minibatch Loss=6525256813955252224.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
St

Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 8676, Minibatch Loss=10092524983495426048.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 8694, Minibatch Loss=10179128016857202688.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 8712, Minibatch Loss=10266267611893334016.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 8730, Minibatch Loss=10353941569580564480.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 8748, Minibatch Loss=10442150989430521856.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 8766, Minibatch Loss=10530899169978089472.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new bat

Iter 9612, Minibatch Loss=15347980557543800832.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 9630, Minibatch Loss=15465013674715906048.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 9648, Minibatch Loss=15582690006190260224.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 9666, Minibatch Loss=15701006253431980032.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 9684, Minibatch Loss=15819966814487576576.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 9702, Minibatch Loss=15939573888380305408.000000
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Start new batch!
Iter 9720, Minibatch Loss=16059828574621794304.000000
Start new 